In [44]:
import pandas as pd
import numpy as np
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

from gensim import corpora, models, similarities

# Linear Models 
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

from tqdm import tqdm


C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)


In [2]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head(10)


,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '...",explanation edits made username hardcore metal...,<class 'dict'>
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"['aww', 'match', 'background', 'colour', 'seem...",aww match background colour seemingly stuck th...,<class 'dict'>
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"['hey', 'man', 'really', 'trying', 'edit', 'wa...",hey man really trying edit war guy constantly ...,<class 'dict'>
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"['make', 'real', 'suggestion', 'improvement', ...",make real suggestion improvement wondered sect...,<class 'dict'>
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page']",sir hero chance remember page,<class 'dict'>
5,5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,"['congratulation', 'well', 'use', 'tool', 'wel...",congratulation well use tool well talk,<class 'dict'>
6,6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,"['cocksucker', 'piss', 'around', 'work']",cocksucker piss around work,<class 'dict'>
7,7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,"['vandalism', 'matt', 'shirvington', 'article'...",vandalism matt shirvington article reverted pl...,<class 'dict'>
8,8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,"['sorry', 'word', 'nonsense', 'offensive', 'an...",sorry word nonsense offensive anyway intending...,<class 'dict'>
9,9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,"['alignment', 'subject', 'contrary', 'dulithgow']",alignment subject contrary dulithgow,<class 'dict'>


In [3]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

15.637365818023682


In [4]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,...",explanation edits made username hardcore metal...,"{'explanation': 1, 'edits': 1, 'made': 1, 'use..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[aww, match, background, colour, seemingly, st...",aww match background colour seemingly stuck th...,"{'aww': 1, 'match': 1, 'background': 1, 'colou..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, really, trying, edit, war, guy, con...",hey man really trying edit war guy constantly ...,"{'hey': 1, 'man': 1, 'really': 1, 'trying': 1,..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[make, real, suggestion, improvement, wondered...",make real suggestion improvement wondered sect...,"{'make': 1, 'real': 1, 'suggestion': 1, 'impro..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]",sir hero chance remember page,"{'sir': 1, 'hero': 1, 'chance': 1, 'remember':..."


In [5]:
# MULTICLASS PREDICTION

In [6]:
classLabel = {
    0: "neutral",
    1: "toxic",
    2 : "severe_toxic",
    3 : "obscene",
    4 : "threat",
    5 : "insult",
    6 : "identity_hate" 
}


In [7]:
# Get texts in Toxic and No Toxic
y = np.empty((len(train['cleanWordsAsText']),),dtype=object)
allTextToxicTrain = dict()
for idx in classLabel:
    if classLabel[idx] != "neutral":
        T = np.where(train[classLabel[idx]] == 1)[0]
        allTextToxicTrain[idx] = T
        for i in T:
            if y[i] is None:
                y[i] = [idx]                
            else:
                y[i].append(idx)
indxsOfNeutralTexts = np.where(y == None) 
y[indxsOfNeutralTexts] = [[0]]
indxsOfNeutralTexts = indxsOfNeutralTexts[0]

allTextsNoToxicTrain = [str(train['cleanWordsAsText'][x]) for x in indxsOfNeutralTexts]

idxList = []
for i in allTextToxicTrain.keys():
    #allTextToxicTrain[i] = [str(train['cleanWordsAsText'][j]) for j in allTextToxicTrain[i]]
    idxList = np.unique(np.append(idxList, allTextToxicTrain[i]))
allTextToxicTrain = [str(train['cleanWordsAsText'][j]) for j in idxList]

In [8]:
# Test clasification
test = pd.read_csv('../data/processed/' + nameTestCSV + '.csv', encoding='utf-8')
test.head()

,Unnamed: 0,id,comment_text,listOfCleanWords,cleanWordsAsText
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,"['yo', 'bitch', 'ja', 'rule', 'succesful', 'ev...",yo bitch ja rule succesful ever whats hating s...
1,1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,"['rfc', 'title', 'fine', 'imo']",rfc title fine imo
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...","['source', 'zawe', 'ashton', 'lapland']",source zawe ashton lapland
3,3,00017563c3f7919a,":If you have a look back at the source, the in...","['look', 'back', 'source', 'information', 'upd...",look back source information updated correct f...
4,4,00017695ad8997eb,I don't anonymously edit articles at all.,"['anonymously', 'edit', 'article']",anonymously edit article


In [9]:
allTrainText = [txt if txt is not np.nan else '' for txt in train['cleanWordsAsText']]
allTestText = [txt if txt is not np.nan else '' for txt in test['cleanWordsAsText']]
X_train = allTrainText
X_test = allTestText
yBinary = MultiLabelBinarizer().fit_transform(y)
y_train = yBinary

In [10]:
columns = ["idExp","numFeatures", "algorithm", "Nfolds", "accuaracy", "logloss", "fmeasure"]
dfTestResults = pd.DataFrame(columns=columns)

In [40]:
#Classifiers

# Logistic Regresion
clfLR = OneVsRestClassifier(LogisticRegression(C=5))

# Linear SVC
clfLSVC = OneVsRestClassifier(LinearSVC(C=5))

# Naive Bayes
clfNB = OneVsRestClassifier(MultinomialNB())

# SGD
clfSGD = OneVsRestClassifier(SGDClassifier(loss="log"))

# Decision Tree
clfDT = OneVsRestClassifier(DecisionTreeClassifier(max_depth=8))





In [12]:
# Vectorizer all text
tdifV = TfidfVectorizer()
X_train_tdif = tdifV.fit_transform(allTrainText)

In [13]:
# BOW
bowFeatures = CountVectorizer(vocabulary=tdifV.get_feature_names())
X_train_bow = bowFeatures.fit_transform(allTrainText)

In [14]:
# Cross validation
Nfolds = 5
kf = KFold(n_splits=Nfolds, random_state=True)
kf.get_n_splits(X_train_tdif)

5

In [15]:
# CV experiments
idExp = 0
X = X_train_tdif

In [16]:
# LINEAR SVC CV exp
name = "Linear SVC"
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0

for train_index, test_index in kf.split(X):
    X_train_cv = X[train_index]
    X_test_cv = X[test_index]
    y_train_cv, y_test_cv = yBinary[train_index], yBinary[test_index]

    clfLSVC.fit(X_train_cv, y_train_cv)
    predicted = clfLSVC.predict(X_test_cv)
    acc = accuracy_score(y_test_cv, predicted)
    fmeausre = f1_score(y_test_cv, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test_cv)
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
dfTestResults.loc[idExp] = [idExp,len(tdifV.get_feature_names()),name,Nfolds,meanAcc,meanLogLoss,meanFmeasure]
print(str(idExp))
idExp += 1

0


In [17]:
# Logistic Regresion CV exp
name = "Logistic Regresion"
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0

for train_index, test_index in kf.split(X):
    X_train_cv = X[train_index]
    X_test_cv = X[test_index]
    y_train_cv, y_test_cv = yBinary[train_index], yBinary[test_index]

    clfLR.fit(X_train_cv, y_train_cv)
    predicted = clfLR.predict(X_test_cv)
    acc = accuracy_score(y_test_cv, predicted)
    fmeausre = f1_score(y_test_cv, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test_cv)
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
dfTestResults.loc[idExp] = [idExp,len(tdifV.get_feature_names()),name,Nfolds,meanAcc,meanLogLoss,meanFmeasure]
print(str(idExp))
idExp += 1

1


In [18]:
# SGD CV exp
name = "SGD"
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0

for train_index, test_index in kf.split(X):
    X_train_cv = X[train_index]
    X_test_cv = X[test_index]
    y_train_cv, y_test_cv = yBinary[train_index], yBinary[test_index]

    clfSGD.fit(X_train_cv, y_train_cv)
    predicted = clfSGD.predict(X_test_cv)
    acc = accuracy_score(y_test_cv, predicted)
    fmeausre = f1_score(y_test_cv, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test_cv)
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
dfTestResults.loc[idExp] = [idExp,len(tdifV.get_feature_names()),name,Nfolds,meanAcc,meanLogLoss,meanFmeasure]
print(str(idExp))
idExp += 1

C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classificat

2


In [19]:
# NB CV exp
name = "NB"
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0

for train_index, test_index in kf.split(X):
    X_train_cv = X[train_index]
    X_test_cv = X[test_index]
    y_train_cv, y_test_cv = yBinary[train_index], yBinary[test_index]

    clfNB.fit(X_train_cv, y_train_cv)
    predicted = clfNB.predict(X_test_cv)
    acc = accuracy_score(y_test_cv, predicted)
    fmeausre = f1_score(y_test_cv, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test_cv)
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
dfTestResults.loc[idExp] = [idExp,len(tdifV.get_feature_names()),name,Nfolds,meanAcc,meanLogLoss,meanFmeasure]
print(str(idExp))
idExp += 1

C:\Anaconda2\envs\py36\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


3


In [20]:
# Decision Tree CV exp
name = "Decision Tree"
meanAcc = 0.0
meanLogLoss = 0.0
meanFmeasure = 0.0

for train_index, test_index in kf.split(X):
    X_train_cv = X[train_index]
    X_test_cv = X[test_index]
    y_train_cv, y_test_cv = yBinary[train_index], yBinary[test_index]

    clfDT.fit(X_train_cv, y_train_cv)
    predicted = clfDT.predict(X_test_cv)
    acc = accuracy_score(y_test_cv, predicted)
    fmeausre = f1_score(y_test_cv, predicted, labels=[0,1,2,3,4,5,6], average=None)
    logloss = log_loss(y_pred=predicted, y_true=y_test_cv)
    meanAcc += acc
    meanLogLoss += logloss
    meanFmeasure += fmeausre
meanAcc = meanAcc / Nfolds
meanLogLoss = meanLogLoss / Nfolds
meanFmeasure = meanFmeasure / Nfolds
dfTestResults.loc[idExp] = [idExp,len(tdifV.get_feature_names()),name,Nfolds,meanAcc,meanLogLoss,meanFmeasure]
print(str(idExp))
idExp += 1

4


In [24]:
dfTestResults.to_excel('report.xls', index=False)

In [29]:
# Get submision from test
def getCSVSubmision(prediction, name):
    columns = ["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
    dfTestPredicted = pd.DataFrame(columns=columns)
    for x in tqdm(range(len(test))):
        dfTestPredicted.loc[x] = [test['id'][x], prediction[x][1], prediction[x][2], prediction[x][3], prediction[x][4], prediction[x][5], prediction[x][6]]

    dfTestPredicted.to_csv('../reports/testPred/'+ str(name) +'.csv',encoding='utf-8', index=False)


In [42]:
# Fit all clasificators with tfid matrix
print("NFeatures = " + str(len(tdifV.get_feature_names())))
tdifVTest = TfidfVectorizer(vocabulary=tdifV.get_feature_names())
X_test_tdif = tdifVTest.fit_transform(X_test)

clfLSVC.fit(X_train_tdif, y_train)
clfLR.fit(X_train_tdif, y_train)
clfSGD.fit(X_train_tdif, y_train)
clfNB.fit(X_train_tdif, y_train)
clfDT.fit(X_train_tdif, y_train)


NFeatures = 158627


OneVsRestClassifier(estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=8,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
          n_jobs=1)

In [33]:
getCSVSubmision(prediction=clfLSVC.predict(X_test_tdif),name="LSVC_TFID")

  3%|▎         | 5274/153164 [00:09<04:38, 530.51it/s]C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 153164/153164 [50:32<00:00, 50.50it/s]


In [34]:
getCSVSubmision(prediction=clfLR.predict_proba(X_test_tdif), name="LR_TFID")

  4%|▍         | 5997/153164 [00:09<04:04, 602.30it/s]C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 153164/153164 [25:47<00:00, 98.99it/s] 


In [48]:
getCSVSubmision(prediction=clfSGD.predict_proba(X_test_tdif), name="SGD_TFID")


  4%|▍         | 5890/153164 [00:09<04:03, 605.56it/s]C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)

  6%|▌         | 8633/153164 [00:14<04:07, 583.39it/s]


 10%|█         | 15933/153164 [00:29<04:16, 534.82it/s]


 14%|█▍        | 22078/153164 [00:44<04:26, 491.83it/s]


 17%|█▋        | 26460/153164 [00:59<04:47, 441.01it/s]


 20%|█▉        | 30261/153164 [01:15<05:05, 402.56it/s]


...

 98%|█████████▊| 150043/153164 [23:35<00:29, 106.02it/s]


 99%|█████████▊| 150902/153164 [23:51<00:21, 105.42it/s]


 99%|█████████▉| 151760/153164 [24:07<00:13, 104.83it/s]


100%|█████████▉| 152616/153164 [24:23<00:05, 104.26it/s]


100%|██████████| 153164/153164 [24:33<00:00, 103.91it/s]

In [45]:
getCSVSubmision(prediction=clfNB.predict_proba(X_test_tdif), name="NB_TFID")


  1%|          | 1071/153164 [00:01<04:19, 585.29it/s]C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)

  5%|▌         | 8185/153164 [00:14<04:24, 548.02it/s]


 10%|▉         | 15154/153164 [00:30<04:33, 504.53it/s]


 14%|█▍        | 21080/153164 [00:45<04:43, 466.13it/s]


 17%|█▋        | 25393/153164 [01:00<05:04, 419.46it/s]


 19%|█▉        | 29003/153164 [01:15<05:24, 382.70it/s]


...

 98%|█████████▊| 150034/153164 [26:53<00:33, 93.00it/s]


 99%|█████████▊| 150924/153164 [27:09<00:24, 92.61it/s]


 99%|█████████▉| 151897/153164 [27:26<00:13, 92.26it/s]


100%|█████████▉| 152827/153164 [27:43<00:03, 91.90it/s]


100%|██████████| 153164/153164 [27:48<00:00, 91.77it/s]

In [47]:
getCSVSubmision(prediction=clfDT.predict_proba(X_test_tdif), name="DT_TFID")


  0%|          | 277/153164 [00:00<03:42, 686.38it/s]C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)

  5%|▌         | 8304/153164 [00:14<04:18, 559.33it/s]


 10%|█         | 15550/153164 [00:29<04:24, 520.14it/s]


 14%|█▍        | 21855/153164 [00:44<04:29, 486.87it/s]


 17%|█▋        | 26273/153164 [01:00<04:49, 437.85it/s]


 20%|█▉        | 30138/153164 [01:15<05:06, 401.62it/s]


...

 98%|█████████▊| 150545/153164 [26:13<00:27, 95.69it/s]


 99%|█████████▉| 151416/153164 [26:30<00:18, 95.20it/s]


 99%|█████████▉| 152288/153164 [26:47<00:09, 94.72it/s]


100%|█████████▉| 153149/153164 [27:05<00:00, 94.24it/s]


100%|██████████| 153164/153164 [27:05<00:00, 94.24it/s]